# Trends.Earth API - Authentication & User Management

This notebook focuses on testing authentication flows and user management features of the Trends.Earth API.

## Table of Contents
1. [Setup and Configuration](#setup)
2. [Authentication Testing](#authentication)
3. [User Profile Management](#profile)
4. [User Administration](#admin)
5. [Token Management](#tokens)

## Setup and Configuration {#setup}

In [ ]:
# Import the shared utilities
import time

from IPython.display import HTML, display
import pandas as pd
from trends_earth_api_utils import (
    TEST_USERS,
    TrendsEarthAPIClient,
    create_user,
    get_current_user,
    update_user_profile,
)

# Configuration
API_URL = "http://localhost:5000"  # Update this for your environment

print(f"🌍 Trends.Earth API URL: {API_URL}")
print(f"📋 Test users available: {list(TEST_USERS.keys())}")

## Authentication Testing {#authentication}

In [ ]:
# Initialize API client
client = TrendsEarthAPIClient(API_URL)

# Test login with regular user
regular_user = TEST_USERS["regular"]
print("🔐 Testing regular user login...")

login_result = client.login(regular_user["email"], regular_user["password"])

if login_result:
    print("✅ Login successful!")
    print(f"   User ID: {client.user_id}")
    print(f"   Access Token: {client.access_token[:20]}...")
    print(f"   Refresh Token: {client.refresh_token[:20]}...")
else:
    print("❌ Login failed - check credentials and API connectivity")

In [ ]:
# Test token refresh functionality
if client.refresh_token:
    print("🔄 Testing token refresh...")

    # Store original token
    original_token = client.access_token

    # Force refresh
    success = client.refresh_access_token()

    if success:
        print("✅ Token refresh successful!")
        print(f"   New token: {client.access_token[:20]}...")
        print(f"   Token changed: {original_token != client.access_token}")
    else:
        print("❌ Token refresh failed")
else:
    print("⚠️  No refresh token available - login first")

## User Profile Management {#profile}

In [ ]:
# Get current user profile
print("👤 Getting current user profile...")
user_profile = get_current_user(client)

if user_profile:
    print("✅ Profile retrieved successfully:")
    profile_df = pd.DataFrame(
        [
            {"Field": k, "Value": str(v) if v is not None else "N/A"}
            for k, v in user_profile.items()
        ]
    )
    display(HTML(profile_df.to_html(index=False)))
else:
    print("❌ Failed to retrieve user profile")

In [ ]:
# Test profile updates
print("✏️  Testing profile updates...")

# Sample update - modify institution
updates = {
    "institution": f"Updated Test Organization - {int(time.time())}",
    "country": "CA",  # Change to Canada
}

update_result = update_user_profile(client, updates)

if update_result:
    print("✅ Profile updated successfully")

    # Verify the update
    updated_profile = get_current_user(client)
    if updated_profile:
        print(f"   New institution: {updated_profile.get('institution', 'N/A')}")
        print(f"   New country: {updated_profile.get('country', 'N/A')}")
else:
    print("❌ Profile update failed")

## User Administration {#admin}

In [ ]:
# Switch to admin user for administration tasks
print("🔐 Switching to admin user...")

# Logout current user
client.logout()

# Login as admin
admin_user = TEST_USERS["admin"]
admin_login = client.login(admin_user["email"], admin_user["password"])

if admin_login:
    print("✅ Admin login successful")
else:
    print("❌ Admin login failed - some admin tests will be skipped")

In [ ]:
# Collect user statistics (admin function)
if admin_login:
    print("📊 Collecting user statistics...")

    # Get user statistics - this would typically come from an admin API endpoint
    # For now, we'll simulate the structure
    try:
        # Try to get user stats from API if available
        response = client.make_request("GET", "/users/stats")
        if response.status_code == 200:
            user_stats = response.json()
        else:
            # Fallback to mock data structure
            user_stats = {
                "total": 50,
                "by_role": {"USER": 45, "ADMIN": 5},
                "by_country": {"US": 20, "CA": 15, "GB": 10, "DE": 5},
            }
            print("⚠️  Using simulated user statistics")
    except Exception as e:
        print(f"⚠️  Could not fetch user statistics: {e}")
        user_stats = {"total": 0, "by_role": {}, "by_country": {}}

    # Extract countries for the summary
    countries = user_stats.get("by_country", {})

    if user_stats:
        print("👥 User Statistics:")
        print(f"   Total Users: {user_stats['total']}")
        if user_stats.get("by_role"):
            roles = user_stats["by_role"]
            print(f"   By Role: {dict(roles)}")
            if countries:
                top_countries = dict(
                    sorted(countries.items(), key=lambda x: x[1], reverse=True)[:5]
                )
                print(f"   Top Countries: {top_countries}")
        else:
            print("   No role breakdown available")
    else:
        print("❌ Failed to get user statistics")
else:
    print("⚠️  Skipping user statistics (not logged in as admin)")

In [ ]:
# Test user creation (admin function)
if admin_login:
    print("👤 Testing user creation...")

    # Generate unique user data
    timestamp = int(time.time())
    new_user_data = {
        "email": f"test_user_{timestamp}@example.com",
        "password": "TestPassword123!",
        "name": f"Test User {timestamp}",
        "country": "US",
        "institution": "Test Institution",
        "role": "USER",
    }

    created_user = create_user(client, new_user_data)

    if created_user:
        print("✅ User created successfully:")
        print(f"   ID: {created_user.get('data', {}).get('id', 'N/A')}")
        print(f"   Email: {new_user_data['email']}")
        print(f"   Name: {new_user_data['name']}")
    else:
        print("❌ User creation failed")
else:
    print("⚠️  Skipping user creation (not logged in as admin)")

## Token Management {#tokens}

In [ ]:
# Test multiple login sessions and token handling
print("🔐 Testing multiple authentication scenarios...")

# Scenario 1: Multiple clients with same user
client2 = TrendsEarthAPIClient(API_URL)
regular_user = TEST_USERS["regular"]

print("\n1️⃣  Testing multiple sessions for same user:")
login1 = client.login(regular_user["email"], regular_user["password"])
login2 = client2.login(regular_user["email"], regular_user["password"])

if login1 and login2:
    print("✅ Multiple sessions created successfully")
    print(f"   Client 1 token: {client.access_token[:20]}...")
    print(f"   Client 2 token: {client2.access_token[:20]}...")
    print(f"   Tokens different: {client.access_token != client2.access_token}")
else:
    print("❌ Multiple session test failed")

In [ ]:
# Test token expiration and automatic refresh
print("\n2️⃣  Testing automatic token refresh on API calls:")

if client.access_token:
    # Make a request that should work
    try:
        response = client.make_request("GET", "/user/me")
        if response.status_code == 200:
            print("✅ API request with valid token successful")
        else:
            print(f"⚠️  API request returned status: {response.status_code}")
    except Exception as e:
        print(f"❌ API request failed: {e}")

    # Simulate token expiration by corrupting the token
    print("\n🔄 Simulating token expiration...")
    original_token = client.access_token
    client.access_token = "invalid_token"
    client.session.headers.update({"Authorization": "Bearer invalid_token"})

    try:
        response = client.make_request("GET", "/user/me")
        if response.status_code == 200:
            print("✅ Automatic token refresh worked!")
            print(f"   New token: {client.access_token[:20]}...")
        else:
            print(f"⚠️  Request failed with status: {response.status_code}")
    except Exception as e:
        print(f"❌ Automatic refresh failed: {e}")
else:
    print("⚠️  No access token available")

In [ ]:
# Test logout functionality
print("\n3️⃣  Testing logout functionality:")

if client.refresh_token:
    print(f"Before logout - Refresh token: {client.refresh_token[:20]}...")

    # Logout
    logout_success = client.logout()

    if logout_success:
        print("✅ Logout successful")
        print(f"   Access token cleared: {client.access_token is None}")
        print(f"   Refresh token cleared: {client.refresh_token is None}")
        print(f"   User ID cleared: {client.user_id is None}")

        # Test that API calls now fail
        try:
            response = client.make_request("GET", "/user/me")
            if response.status_code == 401:
                print("✅ API correctly rejects requests after logout")
            else:
                print(f"⚠️  Unexpected response after logout: {response.status_code}")
        except Exception as e:
            print(f"⚠️  API request after logout failed: {e}")
    else:
        print("❌ Logout failed")
else:
    print("⚠️  No refresh token to logout")

In [ ]:
# Summary and cleanup
print("\n📊 AUTHENTICATION & USER MANAGEMENT TEST SUMMARY")
print("=" * 60)
print("✅ Authentication flow tested")
print("✅ Token refresh mechanism tested")
print("✅ User profile management tested")
print("✅ User administration features tested")
print("✅ Multiple session handling tested")
print("✅ Logout functionality tested")

# Cleanup any remaining sessions
if client2 and client2.refresh_token:
    client2.logout()
    print("\n🧹 Cleaned up test sessions")

print("\n🎉 Authentication and User Management tests completed!")